In [1]:
! pip install rank_bm25

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
from rank_bm25 import BM25Okapi

In [3]:
import pandas as pd

df=pd.read_csv('CompleteMainText.csv')
df_just_text=df[['DocID','Text','URL']]
# print(df_just_text)
df_just_text['Text'] = df_just_text['Text'].str.replace("\n"," ")
corpus = df_just_text['Text'].tolist()
print(len(corpus))
# corpus['Text']=corpus['Text'].tolist()

198


<ipython-input-3-a151e8a71717>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_just_text['Text'] = df_just_text['Text'].str.replace("\n"," ")


In [4]:
# df_corpus_token['Text'] = [string.replace("\n"," ") for string in df_just_text['Text']]
# print(df_corpus_token['Text'][0])

In [5]:
tokenized_corpus = [doc.split(" ") for doc in corpus]

In [6]:
df_tokenized_corpus=df_just_text.copy()
df_tokenized_corpus['Text'] = [doc.split(" ") for doc in df_just_text['Text']]
tokenized_corpus = df_tokenized_corpus['Text'].tolist()
# df_tokenized_corpus['Text'][0]
# df_tokenized_corpus['Text'][0]

In [7]:
bm25 = BM25Okapi(tokenized_corpus)

In [8]:
# df_corpus=df_just_text.copy()
# df_corpus_token=df_just_text.copy()
# df_corpus_token['Text'] = [doc.replace("\n"," ").split(" ") for doc in df_corpus['Text']]

In [9]:
# print(df_corpus_token)

In [10]:
# bm25 = BM25Okapi(df_corpus_token['Text'])

In [11]:
query_list = ["fidgety child unable to sit still",
"child is forgetful in daily activities",
"child disregards rules",
"child not falling asleep in own bed",
"bedwetting in school age children",
"my child worries a lot",
"my child argues with me and screams",
"defiant argumentative behavior in child",
"potty training tips for toddlers",
"handling toddler tantrums",
"child is distracted and cannot focus",
"destructive and aggressive behavior in child"]

tokenized_query=[]
for query in query_list:
  tokenized_query.append(query.split(" "))

In [12]:
# tokenized_query

In [13]:
# bm25.get_top_n(tokenized_query[0], corpus, n=5)

In [14]:
# doc_scores = bm25.get_scores(tokenized_query)

In [15]:
# top_text=bm25.get_top_n(tokenized_query, corpus, n=2)

In [16]:
# doc_index=df_corpus_token[df_corpus_token['Text'] == s, 'DocID'].iloc[0]

In [12]:

final_document_List=[]
diction={}

for a,single_query in enumerate(tokenized_query):
  top_text=bm25.get_top_n(single_query, corpus, n=10)
  # print(top_text)
  # print(df_corpus_token['Text']==top_text[0])
  # mask = df_corpus_token['Title'].apply(lambda x: set(top_text[0]).issubset(set(x)) if isinstance(x, list) else False)
  # doc_Index = df_corpus_token[mask]['DocID']
  # print(doc_Index)

  if query_list[a] not in diction:
        diction[query_list[a]] = []
  
  doc_scores = bm25.get_scores(single_query)
  final_score=sorted(doc_scores)
  for i,s in enumerate(top_text):
    # print(s)
    # print(df_corpus[df_corpus_token['Text'] == s, 'DocID'].iloc[0])
    index=corpus.index(s)
    # print("IndexValuue is->"+ str(index))
    # print("DFID is ->"+str(df_just_text.iloc[index]['DocID']))
    # index = df_just_text.loc[df_just_text['Text']==s].index.tolist()[0]
    # index = df_just_text.loc[df_just_text['Text']==s]['DocID'].item()

    # print(index)
    doc_index=df_just_text.iloc[index]['DocID']
    url=df_just_text.iloc[index]['URL']
    # if doc_index>20:
    #   doc_index+=2
    # elif doc_index<20 and doc_index>8:
    #   doc_index+=1

    # doc_index=df_corpus_token[df_corpus_token['Text'] == s, 'DocID'].iloc[0]
    # doc_index=df.loc[df['Title'] == title, 'DocID'].iloc[0]
    title=df.iloc[index]['Title']
    # print("Document ID-> "+str(index))
    # print("Title ->"+str(title))
    score=final_score[-(i+1)]
    diction[query_list[a]].append((doc_index,title,score,url))
    # print("Rank"+str(i)+"DocID-> :"+str(index)

for query in diction:
  print("QUERY-> "+query)
  for tup in diction[query]:
    print(tup)

QUERY-> fidgety child unable to sit still
(116, "How to Understand Your Child's Temperament - HealthyChildren.org", 9.828414820361512, 'https://healthychildren.org/English/ages-stages/gradeschool/Pages/How-to-Understand-Your-Childs-Temperament.aspx')
(180, "Some Kids Really Are More Difficult—Here's Why It Happens and How to Deal", 9.50575433208952, 'https://www.parents.com/toddlers-preschoolers/discipline/some-kids-really-are-more-difficult-heres-why-it-happens-and-how-to-deal/')
(25, 'Common Time-Out Mistakes and How to Solve Them', 8.977202449465551, 'https://www.parents.com/toddlers-preschoolers/discipline/time-out/common-time-out-mistakes-and-how-to-solve-them/')
(49, 'Toilet Training  (for Parents) - Nemours KidsHealth', 8.58987526267408, 'https://kidshealth.org/en/parents/toilet-teaching.html#catbehavior')
(112, 'Developmental Milestones: 4 to 5 Year Olds - HealthyChildren.org', 8.417528536518333, 'https://healthychildren.org/English/ages-stages/preschool/Pages/Developmental-Mil

After Getting top 10 results from BM25, we will feed it into semantic search

In [13]:
!pip install datasets evaluate transformers[sentencepiece]
!pip install faiss-gpu

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 474.6/474.6 kB 12.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 kB 9.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 68.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 21.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 14.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 12.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 19.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 12.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 16.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 8.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 47.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━

In [14]:
from transformers import AutoTokenizer, AutoModel

model_ckpt = "sentence-transformers/distilbert-base-nli-stsb-mean-tokens"
tokenizer = AutoTokenizer.from_pretrained(model_ckpt)
model = AutoModel.from_pretrained(model_ckpt)

In [15]:
def cls_pooling(model_output):
    return model_output.last_hidden_state[:, 0]

In [16]:
def get_embeddings(text_list):
    encoded_input = tokenizer(
        text_list, padding=True, truncation=True, return_tensors="pt"
    )
    #encoded_input = {k: v.to(device) for k, v in encoded_input.items()}
    model_output = model(**encoded_input)
    return cls_pooling(model_output)

In [17]:
import pickle
embeddings_dataset = pickle.load(open('Distillbert_DocumentLevel_pkl', 'rb'))
embeddings_dataset

Dataset({
    features: ['Title', 'DocID', 'URL', 'Embeddings', 'Text'],
    num_rows: 198
})

In [18]:
df_embeddings = embeddings_dataset.to_pandas()
df_embeddings

,Title,DocID,URL,Embeddings,Text
0,Child Development | Psychology Today,0,https://www.psychologytoday.com/us/basics/chil...,"[0.2803007, 0.23809318, -0.19282328, -0.159848...",Child Development | Psychology TodayUnderstand...
1,Parenting a Child with ADHD | Psychology Today,1,https://www.psychologytoday.com/us/basics/adhd...,"[-0.057591643, 0.2632406, 0.36278155, 0.245281...",Parenting a Child with ADHD | Psychology Today...
2,Raising an Adopted Child | Psychology Today,2,https://www.psychologytoday.com/us/basics/adop...,"[0.30209792, -0.043051302, -0.1418182, -0.2052...",Raising an Adopted Child | Psychology TodayRai...
3,Parenting a Child with Autism | Psychology Today,3,https://www.psychologytoday.com/us/basics/auti...,"[-0.030700075, 0.390369, 0.41788584, -0.278411...",Parenting a Child with Autism | Psychology Tod...
4,Ending Homework Battles | Psychology Today,4,https://www.psychologytoday.com/us/blog/how-ra...,"[0.26746687, 0.18432264, -0.11905984, 0.089509...",Ending Homework Battles | Psychology TodayEndi...
...,...,...,...,...,...
193,My Kid's Toddler Tantrums Are Getting Out of H...,195,https://www.parents.com/parenting/better-paren...,"[0.17019387, -0.038686477, 0.18198003, -0.1769...",My Kid's Toddler Tantrums Are Getting Out of H...
194,10 Ways to Make Feeding Your Toddler 10 Times ...,196,https://www.parents.com/recipes/scoop-on-food/...,"[0.4790843, 0.36725512, -0.06070934, -0.125382...",10 Ways to Make Feeding Your Toddler 10 Times ...
195,Separation and Social Anxiety in Kids,197,https://www.parents.com/toddlers-preschoolers/...,"[0.097834304, -0.50916183, 0.33884814, -0.6288...",Separation and Social Anxiety in KidsRaising K...
196,How Much Does My Kid Need to Eat?,198,https://www.parents.com/toddlers-preschoolers/...,"[0.9504991, 0.4124903, 0.634642, -0.18082817, ...",How Much Does My Kid Need to Eat?Family Life F...


Final Hybrid Search

In [19]:
# import pickle
# embeddings_dataset = pickle.load(open('Distillbert_DocumentLevel_pkl', 'rb'))
# embeddings_dataset
import pandas as pd

result_dict={}
final_df = pd.DataFrame(columns=['Query','rank','Title','DocID','score','URL'])


question = ["fidgety child unable to sit still",
"child is forgetful in daily activities",
"child disregards rules",
"child not falling asleep in own bed",
"bedwetting in school age children",
"my child worries a lot",
"my child argues with me and screams",
"defiant argumentative behavior in child",
"potty training tips for toddlers",
"handling toddler tantrums",
"child is distracted and cannot focus",
"destructive and aggressive behavior in child"]


from datasets import Dataset

# assume df is a pandas dataframe


for i,query in enumerate(question):
  temp_df_embeddings = pd.DataFrame(columns=['Title', 'DocID', 'URL', 'Embeddings', 'Text'])
  
  for individual_doc in diction[query]:
    DocID = individual_doc[0]
    temp_df_embeddings = temp_df_embeddings.append(df_embeddings[df_embeddings['DocID'] == DocID], ignore_index=True)
  # print(temp_df_embeddings)
  embeddings_dataset = Dataset.from_pandas(temp_df_embeddings)
  embeddings_dataset.add_faiss_index(column="Embeddings")

  question_embedding = get_embeddings([question[i]]).cpu().detach().numpy()
  scores, samples = embeddings_dataset.get_nearest_examples("Embeddings", question_embedding, k=5)
  samples_df = pd.DataFrame.from_dict(samples)
  samples_df["scores"] = scores
  samples_df.sort_values("scores", ascending=False, inplace=True)
  samples_df["scores"] = scores
  samples_df.sort_values("scores", ascending=False, inplace=True)
  
  
  print("------------------------")
  print("QUERY-> "+str(i+1)+" "+question[i])
  print("------------------------")

  query=question[i]
  rank=1
  for _, row in samples_df.iterrows():
    title = row.Title
    docID= row.DocID
    qID="q_"+str(i+1)
    dID="d_"+str(docID)
    print(f"TITLE: {row.Title}")
    print(f"SCORE: {row.scores}")
    print(f"DocID: {dID}")
    print(f"URL: {row.URL}")
    print("=" * 50)
    print()
    if qID not in result_dict:
      result_dict[qID]={}
      result_dict[qID][dID]=row.scores
    else:
      result_dict[qID][dID]=row.scores
    final_df.loc[len(final_df)] = [query, rank, title, docID, row.scores,row.URL]

    rank+=1
  


# for a,single_query in enumerate(tokenized_query):
  # top_text=bm25.get_top_n(single_query, corpus, n=10)
  # embeddings_list = []



  # embeddings_dataset=[[get_embeddings([top_text[i]]).cpu().detach().numpy() for i in range(10)]]
  # for result_string in top_text:

    #get the docID for each top_text from the df_just_text dataframe
    # print(df_embeddings[df_embeddings['Text'] == result_string])
    # print(df_embeddings[df_embeddings['Text'].str.contains(result_string)])

    # temp_df_embeddings.append(df_embeddings[df_embeddings['Text'] == result_string], ignore_index=True)
    
    # doc_index=df_just_text.iloc[index]['DocID']
  # print(temp_df_embeddings)
  # for string_result in top_text:
  # for i,question_embedding in enumerate(question):
  #   temp_df_embeddings = pd.DataFrame(columns=['Title', 'DocID', 'URL', 'Embeddings', 'Text'])
  #   question_embedding = get_embeddings([question[i]]).cpu().detach().numpy()
  #   scores, samples = embeddings_dataset.get_nearest_examples("Embeddings", question_embedding, k=5)
  #   samples_df = pd.DataFrame.from_dict(samples)
  #   samples_df["scores"] = scores
  #   samples_df.sort_values("scores", ascending=False, inplace=True)
  #   samples_df["scores"] = scores
  #   samples_df.sort_values("scores", ascending=False, inplace=True)
  #   # print("------------------------")
  #   # print("QUERY-> "+str(i)+" "+question[i])
  #   # print("------------------------")
    
  #   query=question[i]
  #   rank=1
  #   for _, row in samples_df.iterrows():
  #     title = row.Title
  #     docID= df_just_text.iloc[index]['DocID']
  #     qID="q_"+str(i+1)
  #     dID="d_"+str(docID)
  #     # print(f"TITLE: {row.Title}")
  #     # print(f"SCORE: {row.scores}")
  #     # print(f"DocID: {dID}")
  #     # print(f"URL: {row.URL}")
  #     # print("=" * 50)
  #     # print()
  #     if qID not in result_dict:
  #       result_dict[qID]={}
  #       result_dict[qID][dID]=row.scores
  #     else:
  #       result_dict[qID][dID]=row.scores
  #     final_df.loc[len(final_df)] = [query, rank, title, docID, row.scores,row.URL]

  #     rank+=1

<ipython-input-19-717f55f1115f>:34: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  temp_df_embeddings = temp_df_embeddings.append(df_embeddings[df_embeddings['DocID'] == DocID], ignore_index=True)


  0%|          | 0/1 [00:00<?, ?it/s]

------------------------
QUERY-> 1 fidgety child unable to sit still
------------------------
TITLE: Developmental Milestones: 4 to 5 Year Olds - HealthyChildren.org
SCORE: 424.650146484375
DocID: d_112
URL: https://healthychildren.org/English/ages-stages/preschool/Pages/Developmental-Milestones-4-to-5-Year-Olds.aspx

TITLE: How to Understand Your Child's Temperament - HealthyChildren.org
SCORE: 424.650146484375
DocID: d_116
URL: https://healthychildren.org/English/ages-stages/gradeschool/Pages/How-to-Understand-Your-Childs-Temperament.aspx

TITLE: Common Time-Out Mistakes and How to Solve Them
SCORE: 410.2625732421875
DocID: d_25
URL: https://www.parents.com/toddlers-preschoolers/discipline/time-out/common-time-out-mistakes-and-how-to-solve-them/

TITLE: Stop Children From Biting: Strategies and Tips for Parents
SCORE: 399.3033447265625
DocID: d_85
URL: https://www.webmd.com/parenting/stop-children-from-biting#1

TITLE: Stop Children From Biting: Strategies and Tips for Parents
SCORE:

<ipython-input-19-717f55f1115f>:34: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  temp_df_embeddings = temp_df_embeddings.append(df_embeddings[df_embeddings['DocID'] == DocID], ignore_index=True)


  0%|          | 0/1 [00:00<?, ?it/s]

...

  0%|          | 0/1 [00:00<?, ?it/s]

------------------------
QUERY-> 11 child is distracted and cannot focus
------------------------
TITLE: Separation Anxiety  (for Parents) - Nemours KidsHealth
SCORE: 358.15887451171875
DocID: d_52
URL: https://kidshealth.org/en/parents/sep-anxiety.html#catfeelings

TITLE: Nightmares
SCORE: 342.6603088378906
DocID: d_76
URL: https://www.healthychildren.org/English/ages-stages/preschool/Pages/Nightmares-and-Night-Terrors.aspx

TITLE: Problematic Toilet Behaviors  - HealthyChildren.org
SCORE: 341.1484375
DocID: d_103
URL: https://healthychildren.org/English/ages-stages/toddler/toilet-training/Pages/Problematic-Toilet-Behaviors.aspx

TITLE: Nightmares
SCORE: 341.1484375
DocID: d_76
URL: https://www.healthychildren.org/English/ages-stages/preschool/Pages/Nightmares-and-Night-Terrors.aspx

TITLE: How to Understand Your Child's Temperament - HealthyChildren.org
SCORE: 327.1125183105469
DocID: d_116
URL: https://healthychildren.org/English/ages-stages/gradeschool/Pages/How-to-Understand-Your-

<ipython-input-19-717f55f1115f>:34: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  temp_df_embeddings = temp_df_embeddings.append(df_embeddings[df_embeddings['DocID'] == DocID], ignore_index=True)


  0%|          | 0/1 [00:00<?, ?it/s]

------------------------
QUERY-> 12 destructive and aggressive behavior in child
------------------------
TITLE: Parenting a Child with ADHD | Psychology Today
SCORE: 451.8829345703125
DocID: d_1
URL: https://www.psychologytoday.com/us/basics/adhd/parenting-adhd-child

TITLE: Strategies Parents and Kids Can Use to Help Prevent Bullying
SCORE: 451.8829345703125
DocID: d_59
URL: https://childrensmd.org/browse-by-topic/parenting/strategies-parents-and-kids-help-prevent-bullying/

TITLE: Toddler Hitting: Causes and Solutions for Parents
SCORE: 445.3272705078125
DocID: d_174
URL: https://www.parents.com/toddlers-preschoolers/development/behavioral/tough-toddlers-4-common-triggers-for-hitting-and-biting/

TITLE: How to Deal with Toddler Temper Tantrums
SCORE: 407.9287109375
DocID: d_14
URL: https://www.parents.com/toddlers-preschoolers/discipline/tantrum/a-parents-guide-to-temper-tantrums/

TITLE: How to Deal with Toddler Temper Tantrums
SCORE: 389.3549499511719
DocID: d_14
URL: https://www.

In [20]:
index=corpus.index(s)
    # print("IndexValuue is->"+ str(index))
    # print("DFID is ->"+str(df_just_text.iloc[index]['DocID']))
    # index = df_just_text.loc[df_just_text['Text']==s].index.tolist()[0]
    # index = df_just_text.loc[df_just_text['Text']==s]['DocID'].item()

    # print(index)
    doc_index=df_just_text.iloc[index]['DocID']
    url=df_just_text.iloc[index]['URL']
    # if doc_index>20:
    #   doc_index+=2
    # elif doc_index<20 and doc_index>8:
    #   doc_index+=1

    # doc_index=df_corpus_token[df_corpus_token['Text'] == s, 'DocID'].iloc[0]
    # doc_index=df.loc[df['Title'] == title, 'DocID'].iloc[0]
    title=df.iloc[index]['Title']
    # print("Document ID-> "+str(index))
    # print("Title ->"+str(title))
    score=final_score[-(i+1)]
    diction[query_list[a]].append((doc_index,title,score,url))

IndentationError: ignored

In [ ]:
df = embeddings_dataset.to_pandas()
df

In [ ]:
question = ["fidgety child unable to sit still",
"child is forgetful in daily activities",
"child disregards rules",
"child not falling asleep in own bed",
"bedwetting in school age children",
"my child worries a lot",
"my child argues with me and screams",
"defiant argumentative behavior in child",
"potty training tips for toddlers",
"handling toddler tantrums",
"child is distracted and cannot focus",
"destructive and aggressive behavior in child"]

import pandas as pd

final_df = pd.DataFrame(columns=['Query','rank','Title','DocID','score','URL'])

result_dict={}


for i,question_embedding in enumerate(question):
  question_embedding = get_embeddings([question[i]]).cpu().detach().numpy()
  scores, samples = embeddings_dataset.get_nearest_examples("Embeddings", question_embedding, k=5)
  samples_df = pd.DataFrame.from_dict(samples)
  samples_df["scores"] = scores
  samples_df.sort_values("scores", ascending=False, inplace=True)
  samples_df["scores"] = scores
  samples_df.sort_values("scores", ascending=False, inplace=True)
  # print("------------------------")
  # print("QUERY-> "+str(i)+" "+question[i])
  # print("------------------------")
  
  query=question[i]
  rank=1
  for _, row in samples_df.iterrows():
    title=row.Title
    docID=df.loc[df['Title'] == row.Title, 'DocID'].values[0]
    qID="q_"+str(i+1)
    dID="d_"+str(docID)
    print(f"TITLE: {row.Title}")
    print(f"SCORE: {row.scores}")
    print(f"DocID: {dID}")
    print(f"URL: {row.URL}")
    print("=" * 50)
    print()
    if qID not in result_dict:
      result_dict[qID]={}
      result_dict[qID][dID]=row.scores
    else:
      result_dict[qID][dID]=row.scores
    final_df.loc[len(final_df)] = [query, rank, title, docID, row.scores,row.URL]

    rank+=1
#question_embedding

In [ ]:

final_document_List=[]
diction={}

for a,single_query in enumerate(tokenized_query):
  top_text=bm25.get_top_n(single_query, corpus, n=10)
  # print(top_text)
  # print(df_corpus_token['Text']==top_text[0])
  # mask = df_corpus_token['Title'].apply(lambda x: set(top_text[0]).issubset(set(x)) if isinstance(x, list) else False)
  # doc_Index = df_corpus_token[mask]['DocID']
  # print(doc_Index)

  if query_list[a] not in diction:
        diction[query_list[a]] = []
  
  doc_scores = bm25.get_scores(single_query)
  final_score=sorted(doc_scores)
  for i,s in enumerate(top_text):
    # print(s)
    # print(df_corpus[df_corpus_token['Text'] == s, 'DocID'].iloc[0])
    index=corpus.index(s)
    # print("IndexValuue is->"+ str(index))
    # print("DFID is ->"+str(df_just_text.iloc[index]['DocID']))
    # index = df_just_text.loc[df_just_text['Text']==s].index.tolist()[0]
    # index = df_just_text.loc[df_just_text['Text']==s]['DocID'].item()

    # print(index)
    doc_index=df_just_text.iloc[index]['DocID']
    url=df_just_text.iloc[index]['URL']
    # if doc_index>20:
    #   doc_index+=2
    # elif doc_index<20 and doc_index>8:
    #   doc_index+=1

    # doc_index=df_corpus_token[df_corpus_token['Text'] == s, 'DocID'].iloc[0]
    # doc_index=df.loc[df['Title'] == title, 'DocID'].iloc[0]
    title=df.iloc[index]['Title']
    # print("Document ID-> "+str(index))
    # print("Title ->"+str(title))
    score=final_score[-(i+1)]
    diction[query_list[a]].append((doc_index,title,score,url))
    # print("Rank"+str(i)+"DocID-> :"+str(index)

print(diction)

In [25]:
# query='bed wetting'
# for t in diction[query]:
#   print('Score-> '+str(t[2]))
#   print('Title-> '+t[1])
#   print('URL-> '+t[3])

In [26]:
final_df = pd.DataFrame(columns=['Query','rank','Title','DocID','score','URL'])
length=1
for qn,query in enumerate(diction):
  rank=1
  for t in diction[query]:
    docID=t[0]
    title=t[1]
    score=t[2]
    url=t[3]
    final_df.loc[len(final_df)] = [query, rank, title, docID, score,url]
    # final_df.append((query,rank,title,docID,score))
    rank+=1
    length+=1

final_df
  


,Query,rank,Title,DocID,score,URL
0,fidgety child unable to sit still,1,How to Understand Your Child's Temperament - H...,116,9.828415,https://healthychildren.org/English/ages-stage...
1,fidgety child unable to sit still,2,Some Kids Really Are More Difficult—Here's Why...,180,9.505754,https://www.parents.com/toddlers-preschoolers/...
2,fidgety child unable to sit still,3,Common Time-Out Mistakes and How to Solve Them,25,8.977202,https://www.parents.com/toddlers-preschoolers/...
3,fidgety child unable to sit still,4,Toilet Training (for Parents) - Nemours KidsH...,49,8.589875,https://kidshealth.org/en/parents/toilet-teach...
4,fidgety child unable to sit still,5,Developmental Milestones: 4 to 5 Year Olds - H...,112,8.417529,https://healthychildren.org/English/ages-stage...
...,...,...,...,...,...,...
115,destructive and aggressive behavior in child,6,My Kid's Toddler Tantrums Are Getting Out of H...,10,9.946582,https://www.parents.com/parenting/better-paren...
116,destructive and aggressive behavior in child,7,My Kid's Toddler Tantrums Are Getting Out of H...,10,9.946582,https://www.parents.com/parenting/better-paren...
117,destructive and aggressive behavior in child,8,Developmental Milestones: 4 to 5 Year Olds - H...,112,9.941288,https://healthychildren.org/English/ages-stage...
118,destructive and aggressive behavior in child,9,How to Deal with Toddler Temper Tantrums,14,9.901972,https://www.parents.com/toddlers-preschoolers/...


Structuring result dictionary to fit into Evaluation Library

In [27]:
# df = pd.DataFrame(final_df)
temp_result={}
result_dict={}
filtered_df=pd.DataFrame(columns=['Query','DocID','score'])
for i,query in enumerate(query_list):
  filtered_df=final_df.loc[final_df['Query'] == query, ['DocID', 'score']]
  # print(final_df.loc[final_df['Query'] == query, ['DocID', 'score']])
  # Assuming the dataframe is named "df"
  docIDs=filtered_df['DocID'].tolist()
  scores=filtered_df['score'].tolist()
  qID="q_"+str(i+1)
  for j,value in enumerate(docIDs):
    dID="d_"+str(value)
    if qID not in result_dict:
      result_dict[qID]={}
      result_dict[qID][dID]=scores[j]
    else:
      result_dict[qID][dID]=scores[j]
  # selected_rows = final_df[final_df['Query'] == query][['DocID', 'score']]
  # Assuming the dataframe is named "selected_rows"
  # selected_rows['DocID'] = selected_rows['DocID'].apply(lambda x: 'd_' + str(x))
  # result_dict = {temp: selected_rows.to_dict('records')}
  # print(filtered_df)

print(filtered_df)
  # temp_result[temp]=


     DocID      score
110    174  10.763920
111     70  10.555352
112      1  10.148365
113     59  10.069589
114    145  10.068893
115     10   9.946582
116     10   9.946582
117    112   9.941288
118     14   9.901972
119     14   9.901972


In [28]:
print(result_dict['q_2'])

{'d_148': 11.023382677522156, 'd_95': 10.666876972547392, 'd_116': 10.400631913337904, 'd_68': 10.35925569754368, 'd_112': 10.292838790307636, 'd_167': 10.104563130505372, 'd_56': 10.035530035826675, 'd_4': 9.822068118540987, 'd_52': 9.764106861500325, 'd_175': 9.599246844206467}


In [29]:
# for_dic = {}
# for i,query in enumerate(query_list):
#   temp="d_"+str(i)
#   for_dic[temp]=


Validation Start

In [21]:
! pip install ranx

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 97.9/97.9 kB 4.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 25.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.5/311.5 kB 31.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.6/224.6 kB 21.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.6/135.6 kB 13.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 113.3/113.3 kB 9.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Created wheel for pyautocorpus: filename=pyautocorpus-0.1.9-cp310-cp310-linux_x86_64.whl size=189824 sha256=38d7f15599271ea3db2f75c3274b3b8c736ec51579479c1f0bf35191d945866a
  Stored in directory: /root/.cache/pip/wheels/e2/be/43/ec1e16468c393f6e474e3be

In [22]:
from ranx import Qrels, Run

qrels_dict = { "q_1": { "d_65":2, "d_69":2, "d_173":2, "d_1":2, "d_41":2, 
                       "d_15": 1, "d_58": 1, "d_66": 1, "d_69": 1, "d_145": 1, "d_173" : 1},
"q_2": { "d_1":2,"d_41":2,"d_65":2,"d_15":1,"d_58":1,"d_66":1,"d_69":1,"d_145":1,"d_173":1},
"q_3": {"d_6":2,"d_17":2,"d_80":2,"d_120":2,"d_122":2,"d_178":2,
      "d_10":1,"d_13":1,"d_14":1,"d_16":1,"d_18":1,"d_19":1,"d_25":1,"d_79":1,"d_86":1,"d_182":1},
"q_4":{"d_32":2,"d_157":2,"d_159":2,"d_162":2,"d_187":2,
      "d_188":1,"d_33":1,"d_34":1,"d_37":1,"d_52":1,"d_62":1,"d_76":1,"d_109":1,"d_163":1,"d_164":1,"d_189":1},
"q_5":{"d_23":2,"d_26":2,"d_71":2,"d_98":2,"d_183":2,"d_186":2,
      "d_22":1,"d_27":1,"d_29":1,"d_49":1,"d_60":1,"d_63":1},
"q_6":{"d_5":2,"d_8":2,"d_9":2,"d_35":2,"d_50":2,"d_52":2,"d_53":2,"d_71":2,"d_90":2,"d_125":2,"d_153":2,"d_192":2,"d_193":2,"d_194":2,
      "d_34":1,"d_45":1,"d_138":1,"d_162":1,"d_168":1},
"q_7":{"d_178":2,"d_4":2,"d_10":2,"d_14":2,"d_80":2,"d_156":2,"d_195":2,
       "d_16":1,"d_19":1,"d_25":1,"d_36":1,"d_48":1,"d_70":1,"d_86":1,"d_91":1,"d_120":1,"d_152":1,"d_167":1,"d_175":1,"d_178":1,"d_179":1
      },
"q_8":{"d_4":2,"d_6":2,"d_13":2,"d_17":2,"d_19":2,"d_178":2,
      "d_18":1,"d_36":1,"d_70":1,"d_79":1},
"q_9":{"d_22":2,"d_27":2,"d_29":2,"d_49":2,"d_60":2,"d_63":2,"d_75":2,"d_84":2,"d_99":2,"d_100":2,"d_101":2,"d_102":2,"d_103":2,"d_104":2,"d_105":2,"d_149":2,"d_150":2,
      "d_23":1,"d_26":1,"d_46":1,"d_61":1,"d_71":1,"d_88":1,"d_89":1,"d_98":1},
"q_10":{"d_91":2,"d_10":2,"d_14":2,"d_195":2,"d_36":2,"d_48":2,"d_140":2,"d_152":2,"d_154":2,"d_156":2,"d_167":2,"d_179":2,"d_195":2,
        "d_16":1,"d_18":1,"d_25":1,"d_42":1,"d_70":1,"d_95":1,"d_175":1},
"q_11":{"d_41":2,"d_65":2,"d_169":2,
      "d_5":1,"d_15":1,"d_24":1,"d_50":1,"d_54":1,"d_58":1,"d_66":1,"d_69":1,"d_145":1,"d_173":1},
"q_12":{"d_70":2,"d_85":2,"d_174":2,"d_177":2,"d_42":2,"d_96":2,
      "d_10":1,"d_16":1,"d_18":1,"d_25":1}      
 }

run_dict = result_dict


qrels = Qrels(qrels_dict)
run = Run(run_dict)

In [23]:
from ranx import evaluate

# Compute score for a single metric
# evaluate(qrels, run, "ndcg@5")
final_result=evaluate(qrels, run, ["map", "ndcg"])

# Compute scores for multiple metrics at once
# evaluate(qrels, run, ["map@5", "mrr"])
print(final_result)

{'map': 0.10778508771929825, 'ndcg': 0.2394806735442755}


In [ ]:
final_df.to_csv('BM25Result.csv')

Validation END!!!!!!!!!!!!!!!!!!!!